## 1. 질문분류

In [ ]:
import openai
import json
import time  # 재시도 시 딜레이

# OpenAI API 키 설정
openai.api_key = "YOUR_API_KEY"

def classify_question(question):
    """질문을 분석하여 카테고리를 반환하는 GPT"""
    system_prompt = """
    당신은 사용자의 질문을 적절한 카테고리로 분류하는 AI입니다.
    
    가능한 카테고리:
    1. 채용 정보
    2. OCR 분석
    3. 기술 지원
    4. 일반 질문
    
    사용자의 질문을 보고 가장 적절한 카테고리를 하나 선택하세요.

    **출력 형식 (JSON)**
    {"category": "채용 정보"}
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ],
        temperature=0.3,
        max_tokens=50
    )

    return json.loads(response["choices"][0]["message"]["content"])  # JSON 변환


## 2. 데이터검색gpt

In [ ]:
import openai
import os
from dotenv import load_dotenv
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain

import pymysql
import locale
import gradio as gr
import pandas as pd

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

# MySQL 데이터베이스 연결 함수
def connect_to_db():
    return pymysql.connect(
        host='127.0.0.1',
        port=3306,
        user='root',
        passwd='seul0899',
        db='saramin',
        charset="utf8mb4"
    )

# 데이터 가져오기 함수
def fetch_from_saramin_refined():
    connection = connect_to_db()
    cursor = connection.cursor(pymysql.cursors.DictCursor)
    cursor.execute("SELECT * FROM saramin_refined")
    fetch_result = cursor.fetchall()
    connection.close()
    return fetch_result

# 문서 분할 함수
def chunk_document(doc_text, chunk_size=1000, chunk_overlap=100):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_text(doc_text)

# 데이터를 문서로 변환
def convert_to_documents(courses):
    documents = []
    for course in courses:
        doc_text = "\n".join([f"{key}: {value}" for key, value in course.items()])
        chunks = chunk_document(doc_text)
        documents.extend([Document(page_content=chunk) for chunk in chunks])
    return documents

# 벡터 스토어 생성
def create_vector_store(documents):
    embeddings = OpenAIEmbeddings()
    return FAISS.from_documents(documents, embeddings)

# 챗봇 생성 (수정됨)
def create_chatbot(vector_store):
    chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

    # 프롬프트 템플릿 수정 (시스템 & 사용자 메시지 포함)
    prompt_template = PromptTemplate(
        input_variables=["question", "context"],
        template="""
        시스템 프롬프트
        1. 당신은 잡Job의 채용 전문가입니다. 
        2. 사용자의 요구에 맞는 채용 정보를 정확히 제공하세요. 
        3. 제공 정보에는 해당 직무의 '공고제목','공고링크', '기업명' KEY의 VALUE을 반드시 포함하며 이 정보는 수정없이 원본 정보 그대로 전달합니다. 
        4. 'refined_content' key의 value 중 [채용 공고 주요 내용]을 우선적으로 참고하여 답변합니다. [채용 공고 내용]은 사용자의 추가 질문이 있을경우 참고하여 답변하세요
        5. 채용 정보와 관련없는 질문에는 주어진 데이터가 아닌 스스로 답변하되, 자연스럽게 채용 공고와 관련된 질문을 하도록 유도 하세요
        6. 사용자가 인사말을 입력하면 자기 소개와 함께 어떤 정보를 원하는지 물어보세요

        질문: {question}
        
        관련 문서:
        {context}
        """
    )
    return ConversationalRetrievalChain.from_llm(
        llm=chat_model,
        retriever=vector_store.as_retriever(),
        return_source_documents=True,
        verbose=True,
        combine_docs_chain_kwargs={"prompt": prompt_template}  # ✅ 올바르게 수정됨
    )


# 답변 생성 함수 (수정됨)
def get_answer(question, retrieval_chain, chat_history):
    chat_history.append((question, ""))
    
    # 검색 결과가 있는 경우만 문서 활용
    result = retrieval_chain.invoke({"question": question, "chat_history": chat_history})
    
    # 검색된 문서가 없으면 LLM 단독 응답
    if not result.get("source_documents"):
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
        standalone_response = llm.invoke(question)  # LLM만 사용
        return standalone_response, []

    return result.get("answer", ""), result.get("source_documents", [])


# 메인 실행 함수
def main():
    locale.setlocale(locale.LC_ALL, 'ko_KR.UTF-8')
    courses = fetch_from_saramin_refined()
    documents = convert_to_documents(courses)
    vector_store = create_vector_store(documents)
    
    retrieval_chain = create_chatbot(vector_store)
    chat_history = []
    
    def chat_interface(user_input):
        answer, _ = get_answer(user_input, retrieval_chain, chat_history)
        return [(user_input, answer)]
    
    with gr.Blocks() as demo:
        gr.Markdown("<h1 style='text-align: center; color: #6c2dc7;'>잡Job</h1>")
        gr.Image("C:/Users/seul/Downloads/luke-chesser-eICUFSeirc0-unsplash.jpg")
        chatbot = gr.Chatbot()
        user_input = gr.Textbox(label="질문을 입력하세요", placeholder="질문을 입력하세요")
        user_input.submit(chat_interface, inputs=user_input, outputs=chatbot)
    
    demo.launch()

if __name__ == "__main__":
    main()

C:\Users\seul\AppData\Local\Temp\ipykernel_35328\3842493111.py:62: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
C:\Users\seul\AppData\Local\Temp\ipykernel_35328\3842493111.py:67: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(model="gpt-4o", temperature=0.2)
c:\Users\seul\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: 안녕
Assistant: 
Follow Up Input: 안녕
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

        시스템 프롬프트
        1. 당신은 잡Job의 채용 전문가입니다. 
        2. 사용자의 요구에 맞는 채용 정보를 정확히 제공하세요. 
        3. 제공 정보에는 해당 직무의 '공고제목','공고링크', '기업명' KEY의 VALUE을 반드시 포함하며 이 정보는 수정없이 원본 정보 그대로 전달합니다. 
        4. 'refined_content' key의 value 중 [채용 공고 주요 내용]을 우선적으로 참고하여 답변합니다. [채용 공고 내용]은 사용자의 추가 질문이 있을경우 참고하여 답변하세요
        5. 채용 정보와 관련없는 질문에는 주어진 데이터가 아닌 스스로 답변하되, 자연스럽게 채용 공고와 관련된 질문을 하도록 유도 하세요
        6. 사용자가 인사말을 입력하면 자기 소개와 함께 어떤 정보를 원하는지 물어보세요

        질문: 안녕? 어떻게 지내세요?
        
        관련 문서:
        ? 근무 형태
- 3개월 수습기간 이후 정규직으로 전환돼요.

? 근무 장소
- 서울특별시 마포구 월드컵북로 396 누리꿈스퀘어 연구개발
타워 603호 (주)에이

## 3. 질문-답변 검증 GPT

In [ ]:
def verify_answer(question, answer):
    """질문과 답변이 논리적으로 맞는지 검증하는 GPT"""
    system_prompt = """
    당신은 AI 검증 시스템입니다. 사용자의 질문과 AI의 답변이 논리적으로 연결되는지 판단하세요.

    - 질문과 답변이 맞으면: {"verification": "일치"}
    - 질문과 답변이 맞지 않으면: {"verification": "불일치"}

    엉뚱한 답변이거나 맥락이 맞지 않으면 "불일치"로 판단하세요.
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"질문: {question}\n답변: {answer}"}
        ],
        temperature=0.3,
        max_tokens=50
    )

    return json.loads(response["choices"][0]["message"]["content"])  # JSON 변환


# 한번에

In [ ]:
import openai
import os
import json
import logging
import pymysql
from dotenv import load_dotenv
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import gradio as gr

# 로그 설정
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# .env 파일 로드
load_dotenv()

# OpenAI API 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ 1️⃣ 질문 분류 GPT
def classify_question(question):
    """질문을 분석하여 카테고리를 반환하는 GPT"""
    logging.info(f"🟡 [질문 분류] 입력 질문: {question}")

    system_prompt = """
    당신은 사용자의 질문을 적절한 카테고리로 분류하는 AI입니다.
    
    가능한 카테고리:
    1. 채용 정보
    2. 일반 질문
    
    사용자의 질문에 '요청'이 포함되지 않았을시 일반 질문으로 분류하세요.
    질문을 분석하여 가장 적절한 카테고리를 JSON 형식으로 출력하세요.

    **출력 형식 (JSON)**
    {"category": "채용 정보"}
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ],
        temperature=0.3,
        max_tokens=50
    )

    result = json.loads(response["choices"][0]["message"]["content"])
    logging.info(f"🟢 [질문 분류 완료] 카테고리: {result['category']}")
    return result  # JSON 변환

# ✅ 2️⃣ 데이터 검색 GPT
def connect_to_db():
    """MySQL 데이터베이스 연결"""
    return pymysql.connect(
        host='127.0.0.1',
        port=3306,
        user='root',
        passwd='seul0899',
        db='saramin',
        charset="utf8mb4"
    )

def fetch_from_saramin_refined():
    """MySQL에서 채용 정보를 가져옴"""
    connection = connect_to_db()
    cursor = connection.cursor(pymysql.cursors.DictCursor)
    cursor.execute("SELECT * FROM saramin_5_combined_revised_fordb")
    fetch_result = cursor.fetchall()
    connection.close()
    
    logging.info(f"🟢 [DB 데이터 로드 완료] 총 {len(fetch_result)}개 문서 가져옴")
    return fetch_result

def create_vector_store(documents):
    """벡터 스토어를 불러오거나, 없으면 새로 생성하여 저장"""
    embeddings = OpenAIEmbeddings()
    faiss_path = './db/faiss'  # 벡터 스토어 경로

    if os.path.exists(faiss_path):
        logging.info("🟡 [벡터 스토어 로드] 기존 벡터 스토어를 불러옵니다.")
        store = FAISS.load_local(faiss_path, embeddings, allow_dangerous_deserialization=True)
    else:
        logging.info("🔵 [벡터 스토어 생성] 새로운 벡터 스토어를 만듭니다.")
        store = FAISS.from_documents(documents, embeddings)
        store.save_local(faiss_path)
        logging.info("🟢 [벡터 스토어 저장 완료] 새로운 벡터 스토어를 저장했습니다.")

    logging.info(f"🟢 [벡터 스토어 준비 완료] 문서 개수: {len(documents)}")
    return store

def create_chatbot(vector_store):
    """채용 정보 기반 챗봇 생성"""
    chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

    prompt_template = PromptTemplate(
        input_variables=["question", "context"],
        template="""
        시스템 프롬프트:
        1. 당신은 잡Job의 채용 전문가입니다. 사용자의 요구에 맞는 채용 정보를 제공하세요.
        2. 채용 공고에 대한 정보를 제공할 시 다음 형식을 유지하세요 (최대 3개 공고 제공).
        - 공고 제목: (공고제목)
        - 공고 링크: (공고링크)
        - 기업명: (기업명)
        - 주요 업무: (업무 설명)
        - 자격 요건: (필수 자격 요건)

        3. 관련 없는 질문에는 자연스럽게 채용 정보 질문을 유도하세요.

        질문: {question}
        관련 문서: {context}
        """
    )

    return ConversationalRetrievalChain.from_llm(
        llm=chat_model,
        retriever=vector_store.as_retriever(),
        return_source_documents=True,
        verbose=True,
        combine_docs_chain_kwargs={"prompt": prompt_template}
    )

def get_answer(question, retrieval_chain):
    """채용 정보를 검색하여 응답 생성"""
    result = retrieval_chain.invoke({"question": question, "chat_history": []})  # chat_history 추가
    return result.get("answer", ""), result.get("source_documents", [])



# ✅ Gradio 챗봇 인터페이스 추가
def chat_interface(user_input):
    """사용자 입력을 받아 채용 정보 검색 후 답변"""
    category_result = classify_question(user_input)
    category = category_result["category"]

    # 데이터베이스에서 채용 정보 불러오기
    courses = fetch_from_saramin_refined()
    documents = convert_to_documents(courses)
    vector_store = create_vector_store(documents)
    retrieval_chain = create_chatbot(vector_store)

    # 채용 정보 관련 질문일 경우 DB 검색 수행
    if category == "채용 정보":
        answer, _ = get_answer(user_input, retrieval_chain)
    else:
        # 일반 질문은 OpenAI GPT를 활용하여 답변
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 채용 전문 AI 챗봇입니다. 채용 관련 질문에 답변하고, 그 외의 질문에는 한 줄의 간략한 답변을 제공합니다."},
                {"role": "user", "content": user_input}
            ],
            temperature=0.2,
            max_tokens=200
        )
        answer = response["choices"][0]["message"]["content"]

    return [(user_input, answer)]

# ✅ Gradio UI 설정
with gr.Blocks() as demo:
    gr.Markdown("<h1 style='text-align: center; color: #6c2dc7;'>잡Job 챗봇</h1>")
    gr.Image("C:/Users/seul/Downloads/luke-chesser-eICUFSeirc0-unsplash.jpg")
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(label="질문을 입력하세요", placeholder="채용 관련 질문을 입력하세요")

    user_input.submit(chat_interface, inputs=user_input, outputs=chatbot)

# ✅ 실행
if __name__ == "__main__":
    demo.launch()


2025-03-22 16:57:38,362 - INFO - 🟡 [질문 분류] 입력 질문: 데이터분석가가 되고 싶은데 어떤 스펙이 필요해??
2025-03-22 16:57:39,047 - INFO - 🟢 [질문 분류 완료] 카테고리: 일반 질문


현재는 채용 정보 관련 질문만 지원합니다.
